In [10]:
import pybaseball as bb
import pandas as pd
import numpy as np

Returns the batting average of each position  from 2019 season

Batter must have a min of 50 at-bats (AB) 


IDEA: plot positonal batting average over time

In [189]:
# initialize global variables
start_dt = '2019-03-19'
end_dt = '2019-10-31'

def get_position(name: str, year: int = 2019) -> str:
    soup = bb.split_stats.get_split_soup(player_ids[0], year)
    info = bb.split_stats.get_player_info(player_ids[0], soup)
    return info['Position']

In [12]:
# retrieve batting stats
batting = bb.batting_stats_range(start_dt, end_dt)
# filter by min 50 AB
batting = batting[batting['AB'] > 50]

In [13]:
# get list of (last_name, first_name) tuples of all players
players = list(batting['Name'])
players = [name.split() for name in players]
players = [(name[1], name[0]) for name in players]

In [127]:
# retrieve player IDs of each player in players
player_ids = bb.player_search_list(players)
# trim player_ids for only ids that are active during 2019
player_ids = player_ids[player_ids['mlb_played_first'] <= 2019]
player_ids = player_ids[player_ids['mlb_played_last'] >= 2019]

# player_ids = list(player_ids['key_bbref'])

I now have every player's id, batting average, and position

TODO: Group the players by position and average the batting average

In [174]:
def get_position(player_id: int, year: int = 2019) -> str:
    soup = bb.split_stats.get_split_soup(player_id, year)
    info = bb.split_stats.get_player_info(player_id, soup)
    return info['Position']

In [128]:
id_names = []
for i in range(len(batting)):
    name = batting.iloc[i]['Name']
    id_name = ' '.join(name.split()[:2]).lower()
    id_names.append(id_name)

In [172]:
positions = []
for i in range(len(batting)):
    if i % 50 == 0:
        print(f"{i}/{len(batting)}")
    name = batting.iloc[i]['id_name']
    player_id = player_ids.loc[player_ids['Name'] == name]['key_bbref']
    if len(player_id.values) == 0:
        positions.append('N/A')
    else:
        position = get_position(player_id.values[0])
        positions.append(position)

0/541
50/541
100/541
150/541
200/541
250/541
300/541
350/541
400/541
450/541
500/541


In [182]:
for i, position in enumerate(positions):
    p = position.split()
    if len(p) == 1 or p[1] != 'Baseman':
        positions[i] = p[0]
    else:
        positions[i] = ' '.join(p[:2])

In [184]:
batting['Position'] = positions

In [191]:
batting_by_position_2019 = batting.groupby(['Position']).mean()
batting_by_position_2019 = batting_by_position_2019.drop(['N/A', 'Played'])

In [201]:
batting_by_position_2019.to_csv('/Users/mason/w/cs/fun/baseball/data/batting_by_position_2019.csv')

In [202]:
test = pd.read_csv('/Users/mason/w/cs/fun/baseball/data/batting_by_position_2019.csv')

In [203]:
test

,Position,Age,#days,G,PA,AB,R,H,2B,3B,...,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS
0,Catcher,29.026316,603.697368,74.289474,252.565789,225.565789,27.697368,53.631579,10.223684,0.394737,...,3.052632,0.710526,1.618421,5.750000,0.723684,0.289474,0.227592,0.297553,0.386263,0.683737
1,Centerfielder,27.428571,616.738095,93.666667,326.238095,291.095238,43.023810,70.238095,14.690476,1.690476,...,4.642857,1.523810,1.857143,4.976190,8.857143,2.595238,0.233452,0.305119,0.388524,0.693571
2,Designated,28.666667,620.833333,88.833333,360.000000,316.333333,47.500000,81.666667,15.333333,1.000000,...,4.166667,0.000000,2.500000,5.666667,2.166667,0.500000,0.248167,0.333000,0.440000,0.773000
3,First Baseman,28.695652,603.304348,111.500000,423.413043,371.000000,53.565217,93.195652,18.934783,1.000000,...,4.760870,0.021739,2.891304,8.826087,1.695652,0.739130,0.242413,0.324043,0.444435,0.768478
4,Leftfielder,27.518519,603.185185,100.037037,374.740741,336.888889,47.851852,88.370370,19.222222,1.925926,...,3.555556,0.296296,2.629630,6.592593,3.629630,1.148148,0.255741,0.321407,0.444000,0.765481
5,Outfielder,27.111111,610.654321,91.419753,323.654321,288.629630,44.345679,75.518519,15.370370,1.938272,...,4.172840,0.753086,1.679012,5.395062,6.135802,1.604938,0.248259,0.320926,0.424148,0.745037
6,Pinch,27.083333,609.000000,64.250000,147.333333,131.666667,18.916667,31.583333,6.833333,0.750000,...,1.333333,0.333333,0.916667,2.666667,2.333333,0.916667,0.232500,0.299500,0.370083,0.669333
7,Pitcher,27.642857,598.821429,31.928571,68.821429,61.250000,4.357143,8.571429,1.250000,0.035714,...,0.428571,4.392857,0.214286,0.714286,0.178571,0.035714,0.131321,0.163714,0.177357,0.341000
8,Rightfielder,28.354839,610.290323,109.096774,434.580645,384.548387,60.129032,99.580645,19.451613,1.258065,...,4.032258,0.322581,3.161290,8.225806,5.064516,1.129032,0.246161,0.320806,0.454452,0.775226
9,Second Baseman,27.573529,602.764706,91.279412,332.617647,300.588235,41.397059,79.279412,15.838235,1.676471,...,3.470588,0.838235,1.911765,5.397059,4.411765,1.441176,0.253059,0.316456,0.404912,0.721309
